# Datos Temperatura WRF 1.5 Ajustados con Climatología

## Cargar Librerias
Se cargan las librerías esenciales para la manipulación de datos, visualización, cálculos numéricos, manejo de fechas, interacción con la web y operaciones del sistema operativo.

In [1]:
import glob  # Para buscar archivos en un directorio.
import pandas as pd  # Para manipulación de datos.
import matplotlib.pyplot as plt  # Para visualización de datos en 2D.
import numpy as np  # Para cálculos numéricos.
from datetime import datetime, date, timedelta  # Para fechas y horas.
import requests  # Para realizar solicitudes HTTP.
import urllib.request  # Para trabajar con URLs.
import io  # Para manejar entradas/salidas.
import os  # Para interactuar con el sistema operativo.

## Descarga de Datos desde el Servidor para un Rango de Fechas

La función `descargar_datos_desde_servidor` recibe dos parámetros, el `enlace_base` que señala la ubicación de los datos en el servidor, y `fecha_inicio` que marca el punto de partida temporal. La función inicia obteniendo la fecha actual más un día para establecer la fecha final de descarga. Luego, inicializa dos listas vacías: `dataframes` para almacenar los DataFrames (datos) y `file_date` para las fechas de archivo correspondientes. 

Posteriormente, itera a través del rango de fechas desde la fecha de inicio hasta la fecha final. En cada iteración, forma el enlace completo para descargar el archivo CSV correspondiente a cada día en el rango. Se realiza una solicitud HTTP GET para obtener el archivo CSV del servidor, y si la solicitud es exitosa (código de estado 200), los datos se decodifican y se crea un DataFrame de Pandas. 

Se realiza un cálculo para sumar las columnas 1, 2, 3 y 4 del DataFrame, y se agrega una nueva columna llamada "Valor" con el resultado. El DataFrame resultante se agrega a la lista de DataFrames, y el nombre de archivo correspondiente (fecha siguiente) se agrega a la lista de nombres de archivo. La función continúa iterando para el siguiente día hasta llegar a la fecha final.

Finalmente, crea una nueva lista llamada `data_modelo` que contiene subconjuntos de cada DataFrame original con solo las columnas "Estacion" y "Valor". La función devuelve una tupla con `data_modelo` y `file_date`.

In [2]:
def descargar_datos_desde_servidor(enlace_base, fecha_inicio, fecha_fin):
    """
    Función que carga todos los archivos de temperatura pronosticada desde el servidor
    
    Parameters:
        enlace_base (str): Enlace base del servidor donde se encuentran los archivos de temperatura del modelo.
        fecha_inicio (datetime.date): Fecha en la que se iniciará la descarga de los archivos.
        fecha_fin (datetime.date): Fecha en la que se finalizará la descarga de los archivos.

    Returns:
        tuple: Una tupla que contiene dos elementos:
            - data_modelo: Una lista de DataFrames con los datos de temperatura del modelo para cada día del rango.
            - file_date: Una lista de fechas de archivo correspondientes a cada DataFrame descargado en data_modelo.
    """

    # Inicializar listas vacías para almacenar los DataFrames de temperatura pronosticada y los fechas correspondientes.
    data_modelo, file_date = [], []

    fecha_actual = fecha_inicio # Establecer fecha actual como fecha de incio

    # Iterar sobre el rango de fechas
    while fecha_actual < fecha_fin:
        fecha_siguiente = fecha_actual + timedelta(days=1) # Obtener la fecha siguiente

        # Formar el enlace completo para el archivo CSV con la fecha actual y siguiente
        enlace_siguiente = f"{enlace_base}{fecha_actual.strftime('%Y%m%d')}/{fecha_siguiente.strftime('%Y%m%d')}.temperatura.csv"

        # Realizar la solicitud HTTP GET para obtener el archivo CSV
        response = requests.get(enlace_siguiente)

        if response.status_code == 200: # Verificar si la solicitud fue exitosa (código de estado 200)
            csv_data = response.content.decode('utf-8') 
            # Obtener los datos CSV de la respuesta y decodificarlos como texto UTF-8

            # Crear un DataFrame de Pandas a partir de los datos CSV y especificar el encabezado de columnas
            df = pd.read_csv(io.StringIO(csv_data), header=None, names=["Estacion", "Tmax", "Tmin"])

            # Verificar si las columnas "Tmax" y "Tmin" solo contienen valores str
            if df['Tmax'].apply(lambda x: isinstance(x, str)).all() and df['Tmin'].apply(lambda x: isinstance(x, str)).all():
                print(f"Valores en Tmax y Tmin para la fecha {fecha_siguiente} son del tipo str. No se carga el archivo.")
            else:
                # Sustituir los valores str por NaN en las columnas "Tmax" y "Tmin"
                df[['Tmax', 'Tmin']] = df[['Tmax', 'Tmin']].apply(pd.to_numeric, errors='coerce')
                # Imprimir una nota sobre los valores sustituidos por NaN
                replaced_values = df[df.applymap(lambda x: isinstance(x, str))].dropna(how='all')
                
                #Imprimir mensaje si hay valores reemplazados
                if not replaced_values.empty:
                    print(f"Valores sustituidos por NaN en Tmax y Tmin para la fecha {fecha_siguiente}:\n{replaced_values}")
                
                data_modelo.append(df) # Agregar el DataFrame de temperatura pronosticada a la lista de DataFrames
                file_date.append(fecha_siguiente.strftime('%Y%m%d')) # Agregar la fecha de archivo correspondiente
        else:
            #Imprimir un mensaje si el archivo de lluvia pronosticada no se encontró
            print(f"El archivo para la fecha {fecha_siguiente} no se encontró en el servidor.")

        fecha_actual = fecha_siguiente # Actualizar la fecha actual para la próxima iteración del ciclo
    return data_modelo, file_date

### Descarga de Datos desde el Servidor para un Día

La función `obtener_datos_un_dia` permite obtener y procesar los datos de precipitación para un día actual desde un enlace. Recibe como parámetro `enlace_base`, que es la ruta base donde se encuentran los archivos de en el servidor. La función obtiene la fecha actual y la fecha siguiente en formato "YYYYMMDD". Luego, construye el enlace completo para el archivo CSV utilizando estas fechas y el enlace base.

Después, la función lee directamente los datos desde el enlace usando Pandas y agrega un encabezado personalizado para las columnas. Los datos contienen información sobre estaciones y cuatro columnas de precipitación, y se realiza un cálculo para sumar las columnas 1, 2, 3 y 4 del DataFrame para obtener una columna llamada "Valor".

Finalmente, se seleccionan solo las columnas "Estacion" y "Valor" para crear una nueva tabla llamada `data_modelo` que contiene la información relevante. La función devuelve una tupla que contiene `data_modelo`, que es el DataFrame con los datos de precipitación para el día actual, y `file_date`, que es el nombre del archivo correspondiente al día siguiente.

In [3]:
def obtener_datos_un_dia(enlace_base):
    """
    Función que carga el archivo correspondiente a la temperatura pronosticada del día siguiente desde el servidor
    
    Parameters:
        enlace_base (str): El enlace base donde se encuentran los archivos.

    Returns:
        tuple: Una tupla que contiene dos elementos:
            - data_modelo: Un DataFrame con los datos para el día siguiente.
            - file_date: La fecha del archivo correspondiente al día siguiente.
    """

    try:
        # Obtener la fecha actual y la fecha siguiente en formato "YYYYMMDD"
        fecha_actual = date.today().strftime("%Y%m%d")
        fecha_siguiente = (date.today() + timedelta(days=1)).strftime("%Y%m%d")

        # Formar el enlace completo para el archivo de temperatura pronosticada con la fecha actual y siguiente
        enlace_completo = f"{enlace_base}{fecha_actual}/{fecha_siguiente}.temperatura.csv"

        # Realizar la solicitud HTTP GET para obtener el archivo CSV
        response = requests.get(enlace_completo)

        if response.status_code == 200: # Verificar si la solicitud fue exitosa (código de estado 200)
            # Leer los datos directamente desde el enlace y agregar un encabezado personalizado
            encabezado = ["Estacion", "Tmax", "Tmin"]   # Reemplaza con tus etiquetas de columna deseadas
            data_modelo = pd.read_csv(enlace_completo, sep=",", header=None, names=encabezado)

            # Verificar si las columnas contienen solo valores str
            if (data_modelo['Tmax'].apply(lambda x: isinstance(x, str)).all() and data_modelo['Tmin'].apply(lambda x: isinstance(x, str)).all()):
                print(f"Valores en Tmax y Tmin para el día {fecha_actual} son del tipo str. No se carga el archivo.")
            else:
                # Convertir los valores a números y establece los valores que no se pueden convertir en NaN.
                data_modelo[['Tmax', 'Tmin']] = data_modelo[['Tmax', 'Tmin']].apply(pd.to_numeric, errors='coerce')
                # Imprimir una nota sobre los valores sustituidos por NaN
                replaced_values = data_modelo[data_modelo.applymap(lambda x: isinstance(x, str))].dropna(how='all')
                
                #Imprimir mensaje si hay valores reemplazados
                if not replaced_values.empty:
                    print(f"Valores sustituidos por NaN para el día {fecha_actual}:\n{replaced_values}")

        file_date = fecha_siguiente # Guardar la fecha en una nueva variable    
        return data_modelo, file_date

    # Mensaje de error si no se obtienen archivos
    except requests.exceptions.RequestException as e:
        print(f"Error al obtener el archivo: {e}")
        return None, None

### Carga de Tablas Climatología Mensual (Observada)

La función `cargar_tablas_desde_excel` se encarga de cargar las tablas desde un archivo Excel y convertir los nombres de las columnas a cadenas de texto.

La función devuelve un diccionario que contiene las tablas cargadas. Las claves de este diccionario son los nombres de las hojas de Excel proporcionadas en la lista `hojas_excel`, y los valores son DataFrames de Pandas que representan las tablas con los datos.

El proceso que sigue la función es iterar sobre cada nombre de hoja en la lista `hojas_excel`. Para cada hoja, carga los datos desde el archivo Excel utilizando Pandas. Luego, convierte los nombres de las columnas a cadenas de texto. Posteriormente, agrega la tabla a un diccionario llamado `tablas`, donde la clave es el nombre de la hoja y el valor es el DataFrame que contiene los datos de esa hoja.

Finalmente, la función devuelve el diccionario `tablas` que contiene todas las tablas cargadas desde el archivo Excel, cada una identificada por el nombre de la hoja correspondiente.

In [4]:
def cargar_tablas_desde_excel(archivo_excel, hojas_excel):
    """
    Cargar las tablas de la climatología mensual de la temperatura observada
    
    Parameters:
        archivo_excel (str): Ruta del archivo Excel que contiene las tablas.
        hojas_excel (list): Lista de nombres de las hojas de Excel a cargar.

    Returns:
        dict: Un diccionario que contiene las tablas cargadas. Las claves son los nombres de las hojas y los valores son DataFrames de Pandas.
    """
    # Crear un diccionario para almacenar las tablas de la climatología mensual (temperatura observada) cargadas desde Excel
    tablas = {}

    # Iterar sobre la lista de nombres de hojas de Excel a cargar
    for hoja in hojas_excel:
        # Cargar la tabla desde la hoja de Excel especificada
        tabla = pd.read_excel(archivo_excel, sheet_name=hoja)
        
        # Convertir los nombres de las columnas a cadenas de texto
        tabla.columns = tabla.columns.astype(str)
        
        # Agregar la tabla al diccionario utilizando el nombre de la hoja como clave
        tablas[hoja] = tabla

    return tablas

### Ajuste de Datos (Rango de Fechas) con Climatología Mensual (Observada)

La función comienza creando dos DataFrames vacíos: `tabla_ajuste_tmax` y `tabla_ajuste_tmin`. Estos DataFrames se utilizarán para almacenar los registros de ajuste de temperaturas máximas y mínimas respectivamente. También se reciben los DataFrames de datos (`data_modelo`), las fechas correspondientes (`file_date`), las tablas con límites de temperatura (`tablas`) y la ruta de la carpeta donde se guardarán los resultados (`carpeta`).

La función itera por cada estación en los DataFrames de ajuste (`tabla_ajuste_tmax` y `tabla_ajuste_tmin`). Para cada estación, realiza lo siguiente:
   - Obtiene los límites mínimos y máximos de temperatura para el mes siguiente desde las tablas de límites (`tablas`).
   - Filtra los datos por la estación actual.
   - Evalúa si las temperaturas máximas y mínimas registradas en los datos están por encima o por debajo de los límites de la climatología. 
   - Realiza ajustes en los valores si es necesario, actualizando tanto las tablas de datos originales como las filas de ajuste.

Después de ajustar los valores para todas las estaciones en una iteración, las filas de ajuste `tabla_tmax_fila` y `tabla_tmin_fila` se concatenan a los DataFrames principales de ajuste `tabla_ajuste_tmax` y `tabla_ajuste_tmin`. A medida que avanza en el procesamiento, la función guarda cada Dataframe (`tabla`) en archivos CSV y TXT en la carpeta especificada (`carpeta`).

Finalmente, al completar el procesamiento para todos los DataFrames y fechas, la función guarda y devuelve las tablas de ajuste finales, `tabla_ajuste_tmax` y `tabla_ajuste_tmin`.

In [5]:
def procesar_datos_all(data_modelo, file_date, tablas, carpeta):
    """
    Ajustar los datos de temperatura pronosticada con las tablas de la climatología mensual de temperatura observada
    
    Parameters:
        data_modelo (list): Lista de DataFrames que contienen los datos de temperatura pronosticada.
        file_date (list): Lista de fechas de archivo correspondientes a cada DataFrame en data_modelo.
        tablas (dict): Diccionario con las tablas cargadas de climatología mensual (temperatura observada) desde Excel.
        carpeta (str): Ruta donde se guardarán los archivos de resultados.

    Returns:
        tabla_ajuste_tmax, tabla_ajuste_tmin: DataFrames con los datos procesados y ajustados.
    """

    # Crear una tabla para el ajuste con las columnas adecuadas
    tabla_ajuste_tmax = pd.DataFrame(columns=['FECHA'] + [str(col) for col in data_modelo[0]['Estacion']])
    tabla_ajuste_tmin = pd.DataFrame(columns=['FECHA'] + [str(col) for col in data_modelo[0]['Estacion']])
    
    # Iterar sobre los DataFrames de temperatura pronosticada y las fechas correspondientes
    for tabla, fecha in zip(data_modelo, file_date):
        fecha_dt = datetime.strptime(fecha, "%Y%m%d")  # Convertir fecha a formato de datetime
        mes = fecha_dt.month # Obtener el mes correspondiente

         # Crear una nueva fila con la fecha correspondiente para Tmax y Tmin
        tabla_tmax_fila = pd.DataFrame({"FECHA": [fecha_dt.strftime("%Y%m%d")]})
        tabla_tmin_fila = pd.DataFrame({"FECHA": [fecha_dt.strftime("%Y%m%d")]})

        # Iterar sobre las estaciones en las tabla de ajuste
        for estacion in tabla_ajuste_tmax.columns[1:]:
            estacion = str(estacion)

            # Obtener el valor de climatología mensual de temperatura observada (Tmax y Tmin) para la estación y el mes correspondiente
            valor_tmax_min = tablas['tmax_min'].loc[tablas['tmax_min']["FECHA"] == mes, estacion].values[0]
            valor_tmax_max = tablas['tmax_max'].loc[tablas['tmax_max']["FECHA"] == mes, estacion].values[0]

            valor_tmin_min = tablas['tmin_min'].loc[tablas['tmin_min']["FECHA"] == mes, estacion].values[0]
            valor_tmin_max = tablas['tmin_max'].loc[tablas['tmin_max']["FECHA"] == mes, estacion].values[0]

            # Filtrar las filas de la temperatura pronosticada por estación
            filas_estacion = tabla.loc[tabla["Estacion"].astype(str).str.strip() == estacion]

            # Verificar que la fila para la 'estación' en temperatura pronosticada no esté vacía
            if not filas_estacion.empty:
                
                # Obtener el valor de la temperatura pronosticada (Tmax y Tmin) para la estación y el mes
                valor_tmax_modelo = filas_estacion["Tmax"].iloc[0]
                valor_tmin_modelo = filas_estacion["Tmin"].iloc[0]

                # Ajustar la Tmax pronosticada con los limites de la climatología mensual de Tmax observada
                if valor_tmax_modelo >= valor_tmax_max:
                    tabla_tmax_fila[estacion] = f"{valor_tmax_max}({valor_tmax_modelo})"
                    tabla.loc[filas_estacion.index, "Tmax"] = valor_tmax_max
                elif valor_tmax_modelo < valor_tmax_min:
                    tabla_tmax_fila[estacion] = f"{valor_tmax_min}({valor_tmax_modelo})"
                    tabla.loc[filas_estacion.index, "Tmax"] = valor_tmax_min
                else:
                    tabla_tmax_fila[estacion] = "-" # Si el valor para la estación está dentro de los limites, asignar "-"

                # Ajustar la Tmin pronosticada con los limites de la climatología mensual de Tmin observada
                if valor_tmin_modelo >= valor_tmin_max:
                    tabla_tmin_fila[estacion] = f"{valor_tmin_max}({valor_tmin_modelo})"
                    tabla.loc[filas_estacion.index, "Tmin"] = valor_tmin_max
                elif valor_tmin_modelo < valor_tmin_min:
                    tabla_tmin_fila[estacion] = f"{valor_tmin_min}({valor_tmin_modelo})"
                    tabla.loc[filas_estacion.index, "Tmin"] = valor_tmin_min
                else:
                    tabla_tmin_fila[estacion] = "-" # Si el valor para la estación está dentro de los limites, asignar "-"
            else:
                # Si no hay valor para la estación en la temperatura pronosticada, asignar "x"
                tabla_tmax_fila[estacion] = "x"
                tabla_tmin_fila[estacion] = "x"

        #Agregar la nueva fila a la tabla de ajuste correspondiente
        tabla_ajuste_tmax = pd.concat([tabla_ajuste_tmax, tabla_tmax_fila], ignore_index=True)  # Agregar la nueva fila a la tabla
        tabla_ajuste_tmin = pd.concat([tabla_ajuste_tmin, tabla_tmin_fila], ignore_index=True)  # Agregar la nueva fila a la tabla

        # Guardar cada dataframe de data_modelo (temperatura pronosticada) en formato CSV y TXT
        tabla.to_csv(f"{carpeta}/{fecha}.temperatura.csv", index=False)
        tabla.to_csv(os.path.join(carpeta, f"{fecha}.temperatura.txt"), index=False)

    # Guardar las tabla de ajuste (Tmax y Tmin) en CSV y TXT
    tabla_ajuste_tmax.to_csv(f"{carpeta}/tabla_ajuste_tmax.csv", index=False)
    tabla_ajuste_tmax.to_csv(os.path.join(carpeta, "tabla_ajuste_tmax.txt"), index=False)
    
    tabla_ajuste_tmin.to_csv(f"{carpeta}/tabla_ajuste_tmin.csv", index=False)
    tabla_ajuste_tmin.to_csv(os.path.join(carpeta, "tabla_ajuste_tmin.txt"), index=False)
    
    
    return tabla_ajuste_tmax, tabla_ajuste_tmin

### Ajuste de Datos (Un Día) con Climatología Mensual (Observada)

La función `procesar_datos_un_dia` comienza creando una nueva fila para las temperaturas máximas (`tabla_tmax_fila`) y otra para las temperaturas mínimas (`tabla_tmin_fila`). Estas filas están diseñadas para contener los valores de ajuste y los datos procesados.

La función verifica si la fecha ya existe en los DataFrames de ajuste (`tabla_ajuste_tmax` y `tabla_ajuste_tmin`). Si la fecha ya existe en alguno de los DataFrames, se identifica el índice de la fila correspondiente y la elimina de los DataFrames de ajuste la fila.

Se obtienen los límites mínimos y máximos de temperatura para la estación y el mes desde las tablas de climatología, para Tmax (`tablas['tmax_min']`, `tablas['tmax_max']`) y Tmin (`tablas['tmin_min']`, `tablas['tmin_max']`). La función verifica si hay filas en el DataFrame de datos (`data_modelo`) que coincidan con la estación actual. Si se encuentran filas para la estación, se obtienen los valores de temperatura máxima y mínima del modelo. Luego, se comparan estos valores con los límites máximos y mínimos de la climatología. Si están fuera de estos límites, se ajustan los valores en `data_modelo` y se registran los cambios en las filas de ajuste (`tabla_tmax_fila` y `tabla_tmin_fila`).

Una vez que se han procesado todas las estaciones, las filas de ajuste `tabla_tmax_fila` y `tabla_tmin_fila` se agregan a los DataFrames principales de ajuste `tabla_ajuste_tmax` y `tabla_ajuste_tmin`. Los DataFrames de ajuste y el DataFrame de datos (`data_modelo`) se guardan en archivos CSV y TXT en la carpeta especificada (`carpeta`).

La función retorna los DataFrames de ajuste actualizados `tabla_ajuste_tmax` y `tabla_ajuste_tmin`.

In [1]:
def procesar_datos_un_dia(tabla_ajuste_tmax, tabla_ajuste_tmin, data_modelo, file_date, tablas, carpeta):
    """
    Ajustar los datos de temperatura pronosticada con las tablas de la climatología mensual de temperatura observada
    
    Parameters:
        tabla_ajuste_tmax y tabla_ajuste_tmin (pd.DataFrame): DataFrames que contienen los datos procesados hasta el momento.
        data_modelo (pd.DataFrame): DataFrame con los datos para el día actual.
        file_date (str): Fechas del archivo que contiene los datos para el día actual.
        tablas (dict): Diccionario con las tablas cargadas desde un archivo Excel.
        carpeta (str): Ruta de la carpeta donde se guardarán los archivos CSV con los datos diarios.

    Returns:
        pd.DataFrame: Un DataFrame con los datos procesados y ajustados.
    """

    # Obtener la fecha en formato texto (str)
    fecha_siguiente = (date.today() + timedelta(days=1))
    fecha_siguiente_str = fecha_siguiente.strftime("%Y%m%d")

    # Convertir fecha_siguiente_str a np.int64 para hacer la comparación correctamente
    fecha_siguiente_int = np.int64(fecha_siguiente_str)

    # Verificar si la fecha existe en alguna columna "FECHA" de los DataFrames
    if (tabla_ajuste_tmax["FECHA"] == fecha_siguiente_int).any() or (tabla_ajuste_tmin["FECHA"] == fecha_siguiente_int).any():
        # Si la fecha ya existe, obtener el índice de la fila existente
        idx_fecha_existente_tmax = tabla_ajuste_tmax.loc[tabla_ajuste_tmax["FECHA"] == fecha_siguiente_int].index[0]
        idx_fecha_existente_tmin = tabla_ajuste_tmin.loc[tabla_ajuste_tmin["FECHA"] == fecha_siguiente_int].index[0]

        # Eliminar la fila existente a partir del índice
        if idx_fecha_existente_tmax is not None:
            tabla_ajuste_tmax = tabla_ajuste_tmax.drop(idx_fecha_existente_tmax)

        if idx_fecha_existente_tmin is not None:
            tabla_ajuste_tmin = tabla_ajuste_tmin.drop(idx_fecha_existente_tmin)

    # Crear una nueva fila con la fecha actual
    tabla_tmax_fila = {col: "" for col in tabla_ajuste_tmax.columns}
    tabla_tmax_fila["FECHA"] = fecha_siguiente_str
    
    tabla_tmin_fila = {col: "" for col in tabla_ajuste_tmin.columns}
    tabla_tmin_fila["FECHA"] = fecha_siguiente_str

    for estacion in tabla_ajuste_tmax.columns[1:]:
        # Obtener el valor de climatología mensual de temperatura observada (Tmax) para la estación y el mes correspondiente
        valor_tmax_min = tablas['tmax_min'].loc[tablas['tmax_min']["FECHA"] == fecha_siguiente.month, estacion].values[0]
        valor_tmax_max = tablas['tmax_max'].loc[tablas['tmax_max']["FECHA"] == fecha_siguiente.month, estacion].values[0]

        # Obtener el valor de climatología mensual de temperatura observada (Tmin) para la estación y el mes correspondiente
        valor_tmin_min = tablas['tmin_min'].loc[tablas['tmin_min']["FECHA"] == fecha_siguiente.month, estacion].values[0]
        valor_tmin_max = tablas['tmin_max'].loc[tablas['tmin_max']["FECHA"] == fecha_siguiente.month, estacion].values[0]

        # Verificar si existen filas que coincidan con la estación en data_modelo
        filas_estacion = data_modelo.loc[data_modelo["Estacion"].astype(str) == estacion.strip()]

        # Verificar que la fila para la 'estación' en temperatura pronosticada no esté vacía
        if not filas_estacion.empty:
            # Obtener el valor de temperatura pronosticada (Tmax, Tmin) para la estación y el mes correspondiente
            valor_tmax_descargado = filas_estacion["Tmax"].iloc[0]
            valor_tmin_descargado = filas_estacion["Tmin"].iloc[0]

            # Ajustar la temperatura pronosticada (tmax) con los limites de la climatología mensual de temperatura observada (tmax)
            if valor_tmax_descargado >= valor_tmax_max:
                tabla_tmax_fila[estacion] = f"{valor_tmax_max}({valor_tmax_descargado})"
                data_modelo.loc[filas_estacion.index, "Tmax"] = valor_tmax_max
            elif valor_tmax_descargado < valor_tmax_min:
                tabla_tmax_fila[estacion] = f"{valor_tmax_min}({valor_tmax_descargado})"
                data_modelo.loc[filas_estacion.index, "Tmax"] = valor_tmax_min
            else:
                tabla_tmax_fila[estacion] = "-" # Si el valor está dentro de los limites, asignar "-"

            # Ajustar la temperatura pronosticada (tmin) con los limites de la climatología mensual de temperatura observada (tmin)
            if valor_tmin_descargado >= valor_tmin_max:
                tabla_tmin_fila[estacion] = f"{valor_tmin_max}({valor_tmin_descargado})"
                data_modelo.loc[filas_estacion.index, "Tmin"] = valor_tmin_max
            elif valor_tmin_descargado < valor_tmin_min:
                tabla_tmin_fila[estacion] = f"{valor_tmin_min}({valor_tmin_descargado})"
                data_modelo.loc[filas_estacion.index, "Tmin"] = valor_tmin_min
            else:
                tabla_tmin_fila[estacion] = "-" # Si el valor está dentro de los limites, asignar "-"
        else:
            # Si no hay valor para la estación en data_modelo, asignar "x"
            tabla_tmax_fila[estacion] = "x"
            tabla_tmin_fila[estacion] = "x"

    # Agregar la nueva fila a la tabla de ajuste correspondiente
    tabla_ajuste_tmax = tabla_ajuste_tmax.append(tabla_tmax_fila, ignore_index=True)
    tabla_ajuste_tmin = tabla_ajuste_tmin.append(tabla_tmin_fila, ignore_index=True)

    # Guardar la tablas de ajuste y data_modelo en un archivo CSV y TXT
    tabla_ajuste_tmax.to_csv(f"{carpeta}/tabla_ajuste_tmax.csv", index=False)
    tabla_ajuste_tmax.to_csv(os.path.join(carpeta, "tabla_ajuste_tmax.txt"), index=False)
    
    tabla_ajuste_tmin.to_csv(f"{carpeta}/tabla_ajuste_tmin.csv", index=False)
    tabla_ajuste_tmin.to_csv(os.path.join(carpeta, "tabla_ajuste_tmin.txt"), index=False)
    
    data_modelo.to_csv(os.path.join(carpeta, f"{file_date}.temperatura.txt"), index=False)
    data_modelo.to_csv(f"{carpeta}/{file_date}.temperatura.csv", index=False)

    return tabla_ajuste_tmax, tabla_ajuste_tmin

## Crear y Actualizar Tabla de Datos Ajustados.

La función `crear_tabla_si_no_existe` se encarga de crear una tabla en una carpeta especificada si esta no existe. La función recibe los siguientes parámetros: `carpeta`, `nombre_tabla`, `enlace_base`, `archivo_excel` y `hojas_excel`

La función primero verifica si la carpeta proporcionada (`carpeta`) existe. Si no existe, la crea.

La función construye las rutas completas para los archivos de tabla que se espera crear (`ruta_archivo1` y `ruta_archivo2`) concatenando la carpeta y los nombres de tabla (`nombre_tabla1` y `nombre_tabla2`) con la extensión ".csv". Luego, verifica si tanto `ruta_archivo1` como `ruta_archivo2` existen en la carpeta.


Si los archivos de tabla ya existe, la función asume que los datos se han descargado y procesado previamente. Lee los DataFrames de ajuste desde los archivos CSV existentes utilizando `pd.read_csv()`. Obtienen datos para el siguiente día desde el enlace base usando la función `obtener_datos_un_dia`. Carga las tablas con la función `cargar_tablas_desde_excel` y procesa los datos mediante la función `procesar_datos_un_dia`.

Si ninguno de los archivos de tabla existe (`ruta_archivo1` y `ruta_archivo2`), la función asume que se trata del primer uso y descarga los datos desde un enlace base (`enlace_base`) a partir de una fecha específica (`fecha_inicio`) utilizando la función `descargar_datos_desde_servidor`. También carga las tablas desde un archivo Excel (`archivo_excel`) utilizando las hojas especificadas (`hojas_excel`) con la función `cargar_tablas_desde_excel`. Luego, procesa los datos mediante la función `procesar_datos_all`.

La función no devuelve ningún valor (`None`), ya que su propósito es crear o actualizar la tabla y guardar los datos procesados en el archivo correspondiente.

In [7]:
def crear_tabla_si_no_existe(carpeta, nombre_tabla1, nombre_tabla2, enlace_base, archivo_excel, hojas_excel):
    """
    Verifica si la tabla de ajuste de temperatura existe y corre las demás funciones dependiendo de esto.

    Parameters:
        carpeta (str): Ruta de la carpeta donde se guardará la tabla.
        nombre_tabla (str): Nombre de la tabla a crear.
        enlace_base (str): Enlace base para descargar los datos desde el servidor.
        archivo_excel (str): Ruta del archivo Excel con las tablas.

    Returns:
        None
    """
    # Comprobar si la carpeta existe, si no, la crea
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)

    # Ruta completa del archivo que contendrá la tabla
    ruta_archivo1 = os.path.join(carpeta, f"{nombre_tabla1}.csv")
    ruta_archivo2 = os.path.join(carpeta, f"{nombre_tabla2}.csv")

    # Verificar si ambos archivos existen
    if os.path.exists(ruta_archivo1) and os.path.exists(ruta_archivo2):
        # Si ambos archivos existen, cargar las tablas y procesar el día siguiente
        tabla_ajuste_tmax = pd.read_csv(ruta_archivo1)
        tabla_ajuste_tmin = pd.read_csv(ruta_archivo2)
        #Función que carga el archivo correspondiente a la temperatura pronosticada del día siguiente desde el servidor
        data_modelo, file_date = obtener_datos_un_dia(enlace_base)
        #Cargar las tablas de la climatología mensual de la temperatura observada
        tablas = cargar_tablas_desde_excel(archivo_excel, hojas_excel)
        #Ajustar los datos de temperatura pronosticada con las tablas de la climatología mensual de temperatura observada
        procesar_datos_un_dia(tabla_ajuste_tmax, tabla_ajuste_tmin, data_modelo, file_date, tablas, carpeta)
    else:
        #Asignar rango de fechas para cargar archivos de temperatura del modelo
        fecha_inicio = datetime(2023, 7, 11).date()
        fecha_fin = datetime.now().date() + timedelta(days=1)
        #Función que carga todos los archivos de temperatura pronosticada desde el servidor
        data_modelo, file_date = descargar_datos_desde_servidor(enlace_base, fecha_inicio, fecha_fin)
        #Cargar las tablas de la climatología mensual de la temperatura observada
        tablas = cargar_tablas_desde_excel(archivo_excel, hojas_excel)
        #Ajustar los datos de temperatura pronosticada con las tablas de la climatología mensual de temperatura observada
        procesar_datos_all(data_modelo, file_date, tablas, carpeta)


# Uso del método 'crear_tabla_si_no_existe'
# Definición de la ruta de la carpeta donde se almacenarán los datos.
carpeta = r"C:/Users/arias/OneDrive/Documentos/UCR/TFG/Mod_ajuste_w_Clim/Datos/Temperatura"
# Nombres de las tablas a crear o actualizar.
nombre_tabla1 = "tabla_ajuste_tmax"
nombre_tabla2 = "tabla_ajuste_tmin"
# Enlace base al servidor donde se encuentran los datos de temperatura pronosticada.
enlace_base = "https://wrf1-5.imn.ac.cr/modelo/backup/TESIS/"
# Ruta y nombres de las hojas del archivo Excel que contiene la climatología mensual de temperatura observada.
archivo_excel = 'C:/Users/arias/OneDrive/Documentos/UCR/TFG/Climatologia/Observado/Temperatura/Climatologia_Mensual_Observada_Temperatura.xlsx'
hojas_excel = ['tmax_min', 'tmax_max', 'tmin_min', 'tmin_max']
# Llamada al método 'crear_tabla_si_no_existe' con los parámetros definidos.
crear_tabla_si_no_existe(carpeta, nombre_tabla1, nombre_tabla2, enlace_base, archivo_excel, hojas_excel)

C:\Users\arias\AppData\Local\Temp\ipykernel_24580\1474431535.py:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tabla_ajuste_tmax = tabla_ajuste_tmax.append(tabla_tmax_fila, ignore_index=True)
C:\Users\arias\AppData\Local\Temp\ipykernel_24580\1474431535.py:84: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tabla_ajuste_tmin = tabla_ajuste_tmin.append(tabla_tmin_fila, ignore_index=True)
